In [ ]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['tmldb']

In [ ]:
from datetime import datetime

In [ ]:
collection = db['instruments']

In [ ]:
from bson.son import SON
fut_chains = collection.aggregate(
[
   {
      '$lookup':
        {
          "from": "contracts",
          "localField": 'idinstrument',
          "foreignField": 'idinstrument',
          "as": "futures_chains"
        },
   },
   {   
       "$sort": SON([("futures_chains.expirationdate", -1)])
   }
   
   
])

In [ ]:
db.options.create_index([('idcontract', pymongo.ASCENDING),('expirationdate', pymongo.ASCENDING), ('strike', pymongo.ASCENDING),  ])

In [ ]:
import time
start = time.time()

def get_opt_chain(idcontract):
    c = db.options.aggregate([
    {'$match': { 'idcontract' : idcontract }},     
    {'$sort': SON([('expirationdate', 1), ('strikeprice', 1)])},
    {'$group': { "_id" : "$expirationdate",
                "strikes": { "$addToSet": "$strikeprice" }}},

    ])

    expirations_values = []
    for item in c:
        c2 = db.options.aggregate([
            {'$match': { 'idcontract' : idcontract }},     
            {'$match': { 'expirationdate' : item['_id'] }},
            {'$match': { 'strikeprice' : {"$in": item['strikes']} }},     
            {'$group': { "_id" : "$strikeprice",
                        'chain': { "$push": "$$ROOT" }}},
            {'$sort': SON([('_id', 1)])},   ])

        strikes_values = []
        for s in c2:
            #print(s)
            strikes_values.append(s)
            #pprint.pprint(s)
        item['strikes'] = strikes_values

        #pprint.pprint(item)
        expirations_values.append(item)
    
    return expirations_values

import pprint
for instrument in  collection.find():
    fut_chains = []
    for contract in db.contracts.find({'idinstrument': instrument['idinstrument']}).sort('expirationdate'):
        option_chains = []
        
        contract['option_chains'] = get_opt_chain(contract['idcontract'])        
        #pprint.pprint(contract)
        fut_chains.append(contract)
        
    instrument['fut_chains'] = fut_chains
    
    #pprint.pprint(instrument)
    #instrument['_id'] = instrument['cqgsymbol']
    del instrument['_id']
    db.assetindex.insert_one(instrument)
    

    
    
end = time.time()
print(end - start)

In [ ]:
for option in db.options.find({'idcontract': 7079}).sort([('expirationdate', 1)]):
    print(option)

In [ ]:
db.options.create_index([('exchangesymbol', pymongo.ASCENDING)])
    

In [ ]:
sym = db.instruments.find({'exchangesymbol':'CL'}).next()

In [ ]:
db.futures_data.find({'exchangesymbol':'CL'}).next()

In [ ]:
import json
import pickle
import gzip

instr_name = 'EP'
sym = db.instruments.find({'symbol':instr_name}).next()


with gzip.GzipFile(instr_name+'_instrument.pgz', 'w') as f:
    pickle.dump(sym, f)

In [ ]:
with gzip.GzipFile(instr_name+'_instrument.pgz', 'r') as f:
    chain_data = pickle.load(f)

In [ ]:
sym

In [ ]:
import json
import pickle
import gzip

fut_chains = []
for contract in db.contracts.find({
        'idinstrument': sym['idinstrument'], 
        'expirationdate': {'$gt': datetime(2014, 1, 1, 0,0,0)}}).sort('expirationdate').limit(12):    
    fut_chains.append(contract)


with gzip.GzipFile(instr_name+'_futures.pgz', 'w') as f:
#with open('CL.data', 'bw') as f:
    pickle.dump(fut_chains, f)

In [ ]:
fut_chains

In [ ]:
[x['expirationdate'] for x in fut_chains]

In [ ]:
fut_ids = []

for f in fut_chains:
    fut_ids.append(f['idcontract'])

In [ ]:
fut_ids

In [ ]:
import json
import pickle
import gzip

opt_chains = []
for contract in db.options.aggregate([
    {'$match': {'idcontract': 4736}},
    {'$sort': { 'strikeprice': 1} },    
    {'$group': {
        '_id': {'date': '$expirationdate'},
        'chain': {'$push': '$$ROOT'},
        }
    },    
    {'$sort':{"_id.date":1}}   
    ]):
    opt_chains.append(contract)

with gzip.GzipFile(instr_name+'_options.pgz', 'w') as f:
#with open('CL.data', 'bw') as f:
    pickle.dump(opt_chains, f)

In [ ]:
opt_chains

In [ ]:
d = {}

In [ ]:
d['sd']

In [ ]:
db.futures_data.find({'datetime': datetime(2014, 1, 6, 10,15, 0), 'idcontract': 4736})

In [ ]:
db.futures_data.create_index([('idcontract', pymongo.ASCENDING),('datetime', pymongo.ASCENDING)])